# Toronto Neighbourhoods assessment

### Import and install libraries

In [1]:
import pandas as pd
import numpy as np
import requests

print('imported libraries')

imported libraries


In [2]:
!pip install bs4
#!pip install requests

print('b4 installed')

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
b4 installed


In [3]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
print('imported requests')

imported requests


# Invoke URL request

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r=requests.get(url)

# Get HTML content and check

In [5]:
html_Content=r.text
#html_Content

# Use BeautifulSoup to parse html content

In [6]:
soup = BeautifulSoup(html_Content, 'html5lib')
#print(soup.prettify())

In [7]:
tag_object=soup.title
print("tag object:",tag_object)

tag object: <title>List of postal codes of Canada: M - Wikipedia</title>


# Get the required table

In [8]:
table=soup.find('table')
#table

# Logic to read the table elements and convret to dataframe and check

In [9]:
rows=[]
for row in table.findAll('td'):
    text=row.p.text
    if('\n' in text):
        text=text.replace('\n','')
    #print('text=',text)
    code=text[0:3]
    name=row.span.text               
    name=name.replace(' /',',')              
    if('(' in name):
        borough=name.split('(')[0]
        Name=name.split('(')[1].replace(')','')
    else:
        borough=name
        Name=borough  
        #Ignore Not Assigned Boroughs (Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.)    
    if('Not assigned' not in borough):
        #print(code,borough,Name)
        cell1={}
        if(code not in cell1):            
            cell1['PostalCode']=code
            cell1['Borough']=borough
            #If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
            if('Not Assigned' in Name):
                cell1['Neighbourhood']=borough
            else:
                cell1['Neighbourhood']=Name            
            #print('cell1=',cell1)
            rows.append(cell1)
            #print(rows)
        #else:
            #cell1['PostalCode']=code
            #cell1['Borough'].append(Borough)      
            #cell1['Name'].append(Name)
#print(rows)                
df=pd.DataFrame(rows)
df


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
# Use the replacement logic (given) and analyse the dataframe

In [11]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [12]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [13]:
df.describe()

,PostalCode,Borough,Neighbourhood
count,103,103,103
unique,103,15,103
top,M3K,North York,"Dufferin, Dovercourt Village"
freq,1,24,1


In [14]:
df.shape

(103, 3)

In [15]:
!pip install geocoder

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 16.2 MB/s eta 0:00:01


In [38]:
import geocoder # import geocoder


In [39]:
postalcodes=df[['PostalCode']]
#postalcodes.values
boroughs=df[['Borough']]
boroughs

,Borough
0,North York
1,North York
2,Downtown Toronto
3,North York
4,Queen's Park
...,...
98,Etobicoke
99,Downtown Toronto
100,East Toronto Business
101,Etobicoke


In [47]:

for postal_code in postalcodes.values:
    # initialize your variable to None
    lat_lng_coords = None
    print(lat_lng_coords)
    print(postal_code[0])

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        url = 'https://maps.googleapis.com/maps/api/geocode/json'
        params = {'sensor': 'false', 'address': 'North York, CA'}
        # g = geocoder.google('{}, North York'.format(postal_code)) 
        
        lat_lng_coords = None

        # loop until you get the coordinates
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        
        print('g=',g)                      
        r = requests.get(url, params=params)        
        results = r.json()['results']
        print('results=',results)
        location = results[0]['geometry']['location']
        location['lat'], location['lng']
        
        lat_lng_coords = g.latlng
        print(g)        
        if(g is None):
            print('Geocoder is not working')
            lat_lng_coords='NOT NONE'
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print('latitude:',latitude)
    print('longitude:',longitude)


None
M3A
g= <[REQUEST_DENIED] Google - Geocode [empty]>
results= []


IndexError: list index out of range

<[REQUEST_DENIED] Google - Geocode [empty]>


TypeError: 'NoneType' object is not subscriptable

In [48]:
# some how the geocoder is not working. Tried "Google" geocoding but Billing error is received even after setting the billing. Checking with Support. Will invoke GeoCoder once I am able to create APIKey

In [49]:
# The code was removed by Watson Studio for sharing.

In [51]:
"""
for borough in boroughs.values:
    print('borough:',borough[0])
    counter=0
    #g = geocoder.google('North York')
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    endpoint =  base_url+"?address='"+borough[0]+"'&key="+apikey
    #endpoint = f"{base_url}?address={address_or_zipcode}&key={apikey}"
    # see how our endpoint includes our API key? Yes this is yet another reason to restrict the key
    
    print(base_url)
    
    print(endpoint)
    r = requests.get(endpoint)
    
    print(r)
    results = r.json()
    print(results)
    results = r.json()['results'][0]
    lat = results['geometry']['location']['lat']
    lng = results['geometry']['location']['lng']
    latitude = lat
    longitude = lng
    print('latitude=',lng,'  longitude=',lng)
"""

'\nfor borough in boroughs.values:\n    print(\'borough:\',borough[0])\n    counter=0\n    #g = geocoder.google(\'North York\')\n    base_url = "https://maps.googleapis.com/maps/api/geocode/json"\n    endpoint =  base_url+"?address=\'"+borough[0]+"\'&key="+apikey\n    #endpoint = f"{base_url}?address={address_or_zipcode}&key={apikey}"\n    # see how our endpoint includes our API key? Yes this is yet another reason to restrict the key\n    \n    print(base_url)\n    \n    print(endpoint)\n    r = requests.get(endpoint)\n    \n    print(r)\n    results = r.json()\n    print(results)\n    results = r.json()[\'results\'][0]\n    lat = results[\'geometry\'][\'location\'][\'lat\']\n    lng = results[\'geometry\'][\'location\'][\'lng\']\n    latitude = lat\n    longitude = lng\n    print(\'latitude=\',lng,\'  longitude=\',lng)\n'

# Reading from CSV file

In [53]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [54]:
df_data_1.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_data_1

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


# Merge main Neighbourhoods DF and GeoCodes 

In [55]:
mergedDF = pd.merge(df, df_data_1, on='PostalCode')
mergedDF

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [56]:
mergedDF.shape

(103, 5)

In [57]:
mergedDF.describe()

,Latitude,Longitude
count,103.000000,103.000000
mean,43.704608,-79.397153
std,0.052463,0.097146
min,43.602414,-79.615819
25%,43.660567,-79.464763
50%,43.696948,-79.388790
75%,43.745320,-79.340923
max,43.836125,-79.160497
